In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session




In [2]:
# # !export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128
# import os

# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'


In [3]:
#Imports
import os
import sys
import glob
import torch
import timm
import torchvision

import numpy    as np
import datetime as dt
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot   as plt

from PIL               import Image
from torch.utils.data  import Dataset
from torch.autograd    import Variable
from torch.optim       import lr_scheduler

from torch.utils.data  import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision       import transforms, datasets, models
from os                import listdir, makedirs, getcwd, remove
from os.path           import isfile, join, abspath, exists, isdir, expanduser

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_path = "/kaggle/input/ammi-2024-computer-vision/train/train"
test_path = "/kaggle/input/ammi-2024-computer-vision/test/test"
# extraimage_path = join(data_path, "extraimages/extraimages")

In [5]:
# Transformations for both the training and testing data
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

# Do data transforms here, Try many others
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(256),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor()])

test_transforms = transforms.Compose([ transforms.Resize(255),
                                       transforms.CenterCrop(256),
                                       transforms.ToTensor()])

normalize = transforms.Normalize(mean=mean, std=std)

class CassavaDataset(Dataset):
    def __init__(self, path, transform=None):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.transform = transform

        files = []
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])
        self.file_list = files
        files = None

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        classCategory = self.file_list[idx][0]
        im = Image.open(fileName)
        if self.transform:
            im = self.transform(im)
            
        return im.view(3, 256, 256), classCategory
    
train_data = CassavaDataset(train_path, transform=train_transforms)
test_data = CassavaDataset(test_path, transform=test_transforms) 


validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]


# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


train_set = torch.utils.data.DataLoader(train_data, batch_size=32,
                                             sampler=train_sampler)
test_set = torch.utils.data.DataLoader(train_data, batch_size=32,
                                             sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32)

train_len = len(train_set)*32
valid_len = len(test_set)*32

In [6]:
class Classifier(nn.Module):
    def __init__(self, num_classes, weights=models.MobileNet_V2_Weights.DEFAULT):
        super().__init__()
        
        # Load the pre-trained MobileNetV2 model
        self.mobilenet = models.mobilenet_v2(weights=weights)
        
        # Modify the final fully connected layer to match the number of classes
        num_ftrs = self.mobilenet.classifier[1].in_features
        self.mobilenet.classifier[1] = nn.Linear(num_ftrs, num_classes)
    
    def forward(self, x):
        return self.mobilenet(x)

In [7]:
num_classes = 5
model = Classifier(num_classes)
#print(model)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 77.2MB/s]


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.001,momentum= 0.9)

In [16]:
print(model)

Classifier(
  (mobilenet): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 96, kernel_size=(

In [9]:
def train(model, criterion, data_loader1, optimizer, num_epochs):
    """Simple training loop for a PyTorch model.""" 
    model = model.to(device)
    # Make sure model is in training mode.
    #model.train()

    model.train()
    for epoch in range(num_epochs):
        total_correct = 0
        running_loss = 0

        # Training step
        for i, (inputs, labels) in enumerate(data_loader1):
            inputs , labels  = inputs.to(device),labels.to(device)

            output = model(inputs)
            #print(output.logits)
            output_idx = torch.argmax(output,dim =-1)
            total_correct +=(labels==output_idx).sum().item()
            optimizer.zero_grad()
            loss = criterion(output, labels)
            running_loss+=loss.item()*inputs.size(0)
            loss.backward()
            optimizer.step()

        # Print out progress at the end of epoch.
        print(f"Epoch {epoch}, Train Loss: {running_loss/train_len}, Accuracy: {(total_correct/train_len)*100}%")
    print("Finished Training")
    
    

In [10]:
def test(model, test_set):

    with torch.no_grad():
        model.eval()
        total_loss = 0
        total_correct = 0

        # Testing step
        for inputs, labels in test_set:
            inputs , labels  = inputs.to(device),labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss +=loss.item()*inputs.size(0)
            output_idx = torch.argmax(outputs,dim =-1)
            total_correct +=(labels==output_idx).sum().item()

        print(f"Accuracy: {(total_correct/valid_len)*100}%  Loss: {total_loss/valid_len}")

    

In [11]:
num_epochs = 100 #100
train(model, criterion, train_set, optimizer, num_epochs=num_epochs)

Epoch 0, Train Loss: 1.2117931590690998, Accuracy: 50.63820422535211%
Epoch 1, Train Loss: 0.9309834094691865, Accuracy: 65.2068661971831%
Epoch 2, Train Loss: 0.7838512021182499, Accuracy: 71.34683098591549%
Epoch 3, Train Loss: 0.6733582268180457, Accuracy: 76.01232394366197%
Epoch 4, Train Loss: 0.5942497103094635, Accuracy: 79.00528169014085%
Epoch 5, Train Loss: 0.5595888918344404, Accuracy: 79.44542253521126%
Epoch 6, Train Loss: 0.5386096523733626, Accuracy: 80.69982394366197%
Epoch 7, Train Loss: 0.5087624413459758, Accuracy: 82.02024647887323%
Epoch 8, Train Loss: 0.48613857952090844, Accuracy: 83.51672535211267%
Epoch 9, Train Loss: 0.4661049415147535, Accuracy: 83.38468309859155%
Epoch 10, Train Loss: 0.45761181296072373, Accuracy: 83.86883802816901%
Epoch 11, Train Loss: 0.44305669691111943, Accuracy: 84.33098591549296%
Epoch 12, Train Loss: 0.43081878647077043, Accuracy: 85.34330985915493%
Epoch 13, Train Loss: 0.4218558152927689, Accuracy: 85.12323943661971%
Epoch 14, Tra

In [13]:
test(model,test_set)


Accuracy: 85.06944444444444%  Loss: 0.4278522837234454


In [14]:
# Make submission here
sample_path = "/kaggle/input/ammi-2024-computer-vision/sample_submission_file.csv"
import pandas as pd

def test_mode(model, data_loader):
    """Measures the accuracy of a model on a data set.""" 
    # Make sure the model is in evaluation mode.
    preds = []
    model.eval()
    # We do not need to maintain intermediate activations while testing.
    with torch.no_grad():
        
        # Loop over test data.
        for features, _ in data_loader:
          
            # Forward pass.
            output = model(features.to(device))
            
            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)
            preds.append(pred.cpu().data.numpy())
            
    return preds

preds = test_mode(model, test_loader)
preds = [item.item() for sublist in preds for item in sublist]

# Extract filenames from the ImageFolder object
name = [os.path.basename(img_path) for _, _, img_path in test_data.file_list]
sample = pd.read_csv(sample_path)
mapping = {0: 'cmd', 1: 'cbb', 2: 'cbsd', 3: 'healthy', 4: 'cgm'}
new_preds = [mapping[pred] for pred in preds]
sample['Category'] = name
sample['Id'] = new_preds
sample["Category"],sample["Id"] = sample["Id"],sample["Category"]


sample.to_csv('MobileNet13.csv', index=False)
sample.head()

,Category,Id
0,cgm,test-img-1448.jpg
1,cmd,test-img-768.jpg
2,cmd,test-img-3481.jpg
3,cmd,test-img-1475.jpg
4,cgm,test-img-2498.jpg


In [ ]:
4+8